In [ ]:
!python3.11 -m pip install --upgrade pip
!pip3 install opencv-python ultralytics supervision torch

In [ ]:
# imports
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict

## Data Loading + Preprocessing

In [ ]:
# get frames for initializing 
cap = cv.VideoCapture(1)
model = YOLO('yolov8n.pt')
names = model.names
savedFrames = []

itr = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame)

    for result in results:
        save_buffer = []
        box_buffer = []
        boxes = result.boxes.cpu().numpy()
        
        for box, xywh in zip(boxes.cls, boxes.xyxy):
            if names[box] == 'person':
                x1, y1, x2, y2 = map(int,xywh)
                
                # print(x, y, w, h)
                person_roi = frame[y1:y2, x1:x2]
                save_buffer.append(person_roi)
        if len(save_buffer) == 1:
            savedFrames.append(save_buffer[0])
        else:
            break

    detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )

    frame = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=3, text_thickness=3, text_scale=1.5).annotate(scene=frame, detections=detections)
    cv.imshow('Live Video', frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
tmp = "McLovin_"
itr = 0
for frame in savedFrames:
    bgr = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    cv.imwrite("img/rgb_"+tmp + str(itr) + ".png", frame)
    cv.imwrite("img/bgr_"+tmp + str(itr) + ".png", bgr)
    cv.imwrite("img/hsv_"+tmp + str(itr) + ".png", hsv)
    # print(frame)
    itr += 1

## Model Testing

In [ ]:
cap = cv.VideoCapture(1)
model = YOLO('models/segment-hpo/bestMcChimkns.pt')


while True:
    ret, frame = cap.read()

    if not ret:
        break
    
    results = model.track(frame, persist=True)
    annotated_frame = results[0].plot()

    cv.imshow('Live Video',annotated_frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()

## Frame Sampling

In [ ]:
cap = cv.VideoCapture(1)
cacheFrames = []

while True:
    ret, frame = cap.read()

    if not ret:
        break
    
    # results = model(frame)
    # annotated_frame = results[0].plot()
    cacheFrames.append(frame)
    cv.imshow('Live Video',frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
for frame in cacheFrames:
    
    cv.imshow('Live Video',frame)
    if cv.waitKey(1) & 0xFF == 27:  # Press 'Esc' key to exit the loop
        break

cap.release()
cv.destroyAllWindows()